In [ ]:
cd ..

In [ ]:
import os

In [ ]:
import numpy as np

In [ ]:
from tqdm.autonotebook import tqdm

In [ ]:
import echofilter.raw
import echofilter.shardloader

In [ ]:
root_data_dir = echofilter.raw.loader.ROOT_DATA_DIR

In [ ]:
partition = 'train'
partitioning_version = 'firstpass'
dataset = 'mobile'

In [ ]:
max_depth = 70

In [ ]:
transect_pths = echofilter.raw.loader.get_partition_list(
    partition,
    dataset=dataset,
    partitioning_version=partitioning_version,
    root_data_dir=root_data_dir,
    full_path=True,
    sharded=True,
)

In [ ]:
means = []
medians = []

for transect_pth in tqdm(transect_pths):

    try:
        # Check how many segments the transect was divided into
        with open(os.path.join(transect_pth, 'n_segment.txt'), 'r') as f:
            n_segment = int(f.readline().strip())

        for i_seg in range(n_segment):
            transect = echofilter.shardloader.load_transect_from_shards_abs(
                os.path.join(transect_pth, str(i_seg))
            )
            means.append(np.mean(transect['Sv'][:, transect['depths'] <= max_depth]))
            medians.append(np.median(transect['Sv'][:, transect['depths'] <= max_depth]))
    except Exception as ex:
        print('Error loading shard from {}'.format(transect_pth))
        print(ex)

MEAN = np.mean(means)
print('mean = {}'.format(MEAN))
print('mean of medians = {}'.format(np.mean(medians)))

In [ ]:
variances = []

for transect_pth in tqdm(transect_pths):

    try:
        # Check how many segments the transect was divided into
        with open(os.path.join(transect_pth, 'n_segment.txt'), 'r') as f:
            n_segment = int(f.readline().strip())

        for i_seg in range(n_segment):
            transect = echofilter.shardloader.load_transect_from_shards_abs(
                os.path.join(transect_pth, str(i_seg))
            )
            variances.append(np.mean(np.power(transect['Sv'][:, transect['depths'] <= max_depth] - MEAN, 2)))
    except Exception as ex:
        print('Error loading shard from {}'.format(transect_pth))
        print(ex)


VARIANCE = np.mean(variances)
print('variance = {}'.format(VARIANCE))
print('stdev = {}'.format(np.sqrt(VARIANCE)))